# Chapter 7. Movie Recommendation System Web Application

이번 챕터의 목적은 쟝고 프레임워크를 사용해서 행동에서의(in action) 실제적인 추천 시스템의 예제를 설명하는 것이다. 우리는 영화 추천 시스템을 구현할 것인데 각각의 유저는 그들의 선호를 반영한 영화를 제안 받을 것이다. 우리가 사용할 데이터는 942명의 유저에게 50번이상 평가받은 603개의 영화를 이용할 것이다. 추천을 받기 위해 각각의 유저는 어떤 영화에 대한 레이팅을 가지고 있다. 이전 장에서 정보 인출 시스템을 만들었다. 쟝고 어플리케이션의 다른 파트가 논의 될 것인데 settings, models, user login/logout, commands, information retrieval system, recommendation systems, an admin interface and APIs 가 논의 될 것이다. 이 모든것은 저자의 Git hub 에서 볼 수 있다. https://github.com/ai2010/machine_learning_for_the_web/tree/master/chapter_7

Ch6 쟝고의 기본에서 쟝고의특징들에 대해 살펴 봤다.

# Application setup

We create and start Django as usual:

쟝고 프로젝트를 생성하자

```python
django-admin startproject server_movierecsys
```

and from the server_movierecsys folder we start the application:

books_recsys_app 앱을 만든다

```python
python manage.py startapp books_recsys_app
```

Now the settings.py needs to be configured. As we see in Chapter 6, Basics of Django: a simple web framework we set the installed apps, HTML templates, a layout formatting folder, and an SQLite database:

```python
INSTALLED_APPS = (
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'rest_framework_swagger',
    'books_recsys_app',
)

TEMPLATE_DIRS = (
    os.path.join(BASE_DIR, 'templates'),
)
STATIC_URL = '/static/'
STATICFILES_DIRS = ( os.path.join(BASE_DIR, "static"), )
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}

```

Apart from the standard apps, and the rest framework (swagger), the books_recsys_app has been included in the installed apps list.

표준적인 앱들과는 다르게 rest_framework와 swagger(문서화도구인듯) 과 우리가 만든 books_recsys_app 을 installed apps 에 추가해준다


In this case, we need to load data persistently in the memory so that the user experience is improved by not calculating or retrieving data at each user request. To save data or the results of expensive calculations in the memory, we set up the cache system of Django in settings.py:


이 경우에 우리는 유저의 경험을 향상 시키기 위해 메모리에 데이터를 지속시킬 필요가 있다. 비싼 계산결과나 데이터를 저장하기 위해 우리는 쟝고의 캐싱 시스템을 만들었다.


```python
CACHES = {
    'default': {
        'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
        'LOCATION': '/var/tmp/django_cache',
        'TIMEOUT': None,
    }
}
```
We have chosen the File Based Cache cache type stored in /var/tmp/django_cache and a None timeout which means the data in the cache will never expire.

To use the admin interface, we set up the superuser account through the command:

우리는 파일 베이스 chache를 선택했고 그것은 /var/tmp/django_cache에 저장된다 None timeout은 캐시 안의 데이터가 만료되지 않는다는 뜻이다.

아 깜빡했네 설치먼저 해주고
makemigrations - db 스키마 만들어주고
migrate - db 생성 

해줘야합니다

```python
sudo pip install djangorestframework
sudo pip install django-rest-swagger
sudo pip install jsonfield

python manage.py makemigrations
python manage.py migrate

```

슈퍼유저 생성

```python
python manage.py createsuperuser (admin/admin)
````
The application is live at http://localhost:8000/ by typing:

```python
python manage.py runserver
```

# Models

In this application, we need to store the data related to each movie and the movies' ratings from each user of the website. We set up three models:


```python
class UserProfile(models.Model):
    user = models.ForeignKey(User, unique=True)
    array = jsonfield.JSONField()
    arrayratedmoviesindxs = jsonfield.JSONField()
    lastrecs = jsonfield.JSONField()

    def __unicode__(self):
            return self.user.username

    def save(self, *args, **kwargs):
        create = kwargs.pop('create', None)
        recsvec = kwargs.pop('recsvec', None)
        print 'create:',create
        if create==True:
            super(UserProfile, self).save(*args, **kwargs)
        elif recsvec!=None:
             self.lastrecs = json.dumps(recsvec.tolist())
             super(UserProfile, self).save(*args, **kwargs)
        else:
            nmovies = MovieData.objects.count()
            array = np.zeros(nmovies)
            ratedmovies = self.ratedmovies.all()
            self.arrayratedmoviesindxs = json.dumps([m.movieindx for m in ratedmovies])
            for m in ratedmovies:
                array[m.movieindx] = m.value
            self.array = json.dumps(array.tolist())
            super(UserProfile, self).save(*args, **kwargs)
    
class MovieRated(models.Model):
    user = models.ForeignKey(UserProfile, related_name='ratedmovies')
    movie = models.CharField(max_length=100)
    movieindx = models.IntegerField(default=-1)
    value = models.IntegerField()
    
class MovieData(models.Model):
    title = models.CharField(max_length=100)
    array = jsonfield.JSONField()
    ndim = models.IntegerField(default=300)
    description = models.TextField()
    
```

The model MovieData stores the data for each movie: title, description, and vector representation (ndim is the dimension of the vector representation). MovieRated records each movie rated by the user logged in (each object MovieRated is associated with has a UserProfile that utilizes the website). The UserProfile model stores all the users that sign up to the website, so they can rate movies and receive recommendations. Each UserProfile extends the default Django user model by adding the array field, which stores all the movie's ratings from the user, and the recsvec field which stores his last recommendations: the save function is overridden to fill the array field with all the MovieRated objects associated with the user (if the else statement is true), and to fill the lastrecs field with the last recommendations (else if statement). Note that the MovieRated model has a UserProfile foreign key with the related_name equal to ratedmovies: in the save function of the UserProfile model, self.ratedmovies.all() refers to all the RatedMovie objects that have the same UserProfile value. The field arrayratedmoviesindxs on the UserProfile model records all the movies rated by the user and it is used by the API of the application.

To write these data structures on the database we need to run:

MovieData 모델은 각각의 영화 데이터를 저장한다.(title(제목),description(설명), vector representation(벡터 표현?? ndim 은 벡터 표현의 차원수이다) MovieRated 는 로그인 된 유저가 각각의 영화에 대해 내린 평가를 기록한다( 각각의 MovieRated 객체는 UserProfile 와 관계를 맺고 있다.) UserProfile 모델은 웹사이트에 등록한 모든 유저를 저장하고 그들은 영화에 점수를 매기고 추천을 받을 수 있다. 각각의 UserProfile 은 기본 django user model 을 확장해 array 필드를 추가했고 그것은 user로 부터의 모든 영화 평가를 저장한다. 그리고 recsvec 필드는 그의 마지막 추천을 저장한다. save function은 오버라이딩 되어서 모델이 저장되는 것과 동시에 array 필드를 유저와 관계된 movierated 객체로 채운다. (만약 if else 가 true 라면) 그리고 lastrecs 필드를 마지막으로 받은 추천으로 채운다. MovieRated 모델이 UserProfile 과 관계를 맺는 것을 주목하라 relate_name=ratedmovies 라는 이름으로 관계를 맺고 있다. UserProfile 의 save function 안에서 self.ratedmoves.all() 은 한 유저가 매긴 모든 RatedMovie 를 참조한다. arrayratedmoviesindxs 필드는 유저로 부터 점수가 매겨진 모든 영화를 기록하고이것은 api로 이용될 것이다.


어렵다... 뭔소리야

데이터 구조를 바꿨으니 다시 적용해 준다

```python
python manage.py makemigrations
python manage.py migrate
```

잘돼었는지 확인하기 위해 admin을 살펴보자

```python
from django.contrib import admin

import models

# Register your models here.
admin.site.register(models.UserProfile)
admin.site.register(models.MovieRated)
admin.site.register(models.MovieData)
```

# Commands

The commands used in this application are needed to load the data into the memory (cache) and make the user experience fast. Although the movie database is the same used in Chapter 4, Web mining techniques (that is 603 movies rated more than 50 times by 942 users), each movie needs a description to set up an information retrieval system on the movies to rate. The first command we develop takes all the movie titles in the utility matrix used in Chapter 4, Web Mining Techniques and collects the corresponding descriptions from Open Movie Database (OMDb) online service:

commands 는 메모리로 데이터를 불러 오는데 필요하다(캐싱) 즉 유저의 경험을 빠르게 만들어 준다 . 비록 moview database가 CH4 와 같지만 각각의 영화는 평점을 매길 영화의 설명이 필요하다. 첫번째 커맨드는 CH4의 UTILITY MATRIX에서 모든 영화 제목들을 가져온다. 이는 Open Database(OMdb) online sevice와 일치한다.

```python
from django.core.management.base import BaseCommand
import os
import optparse
import numpy as np
import json
import pandas as pd
import requests
class Command(BaseCommand):

    option_list = BaseCommand.option_list + (
            optparse.make_option('-i', '--input', dest='umatrixfile',
                                 type='string', action='store',
                                 help=('Input utility matrix')),   
            optparse.make_option('-o', '--outputplots', dest='plotsfile',
                                 type='string', action='store',
                                 help=('output file')),  
            optparse.make_option('--om', '--outputumatrix', dest='umatrixoutfile',
                                 type='string', action='store',
                                 help=('output file')),            
        )
        
        
    def getplotfromomdb(self,col,df_moviesplots,df_movies,df_utilitymatrix):
        string = col.split(';')[0]
        
        title=string[:-6].strip()
        year = string[-5:-1]      
        plot = ' '.join(title.split(' ')).encode('ascii','ignore')+'. '
        
        url = "http://www.omdbapi.com/?t="+title+"&y="+year+"&plot=full&r=json"
        
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36"}
        r = requests.get(url,headers=headers)
        jsondata =  json.loads(r.content)
        if 'Plot' in jsondata:
            #store plot + title
            plot += jsondata['Plot'].encode('ascii','ignore')

        if plot!=None and plot!='' and plot!=np.nan and len(plot)>3:#at least 3 letters to consider the movie
            df_moviesplots.loc[len(df_moviesplots)]=[string,plot]
            df_utilitymatrix[col] = df_movies[col]
            print len(df_utilitymatrix.columns)

        return df_moviesplots,df_utilitymatrix
    
    def handle(self, *args, **options):
        pathutilitymatrix = options['umatrixfile']
        df_movies = pd.read_csv(pathutilitymatrix)
        movieslist = list(df_movies.columns[1:])

        df_moviesplots = pd.DataFrame(columns=['title','plot'])
        df_utilitymatrix = pd.DataFrame()
        df_utilitymatrix['user'] = df_movies['user']

        for m in movieslist[:]:
            df_moviesplots,df_utilitymatrix=self.getplotfromomdb(m,df_moviesplots,df_movies,df_utilitymatrix)

        outputfile = options['plotsfile']
        df_moviesplots.to_csv(outputfile, index=False)
        outumatrixfile = options['umatrixoutfile']
        df_utilitymatrix.to_csv(outumatrixfile, index=False)
        
```
The command syntax is:
```python
python manage.py --input=utilitymatrix.csv --outputplots=plots.csv –outputumatrix='umatrix.csv'
```

Each movie title contained in the utilitymatrix file is used by the getplotfromomdb function to retrieve the movie's description (plot) from the website http://www.omdbapi.com/ using the requests in the Python module. The descriptions (and titles) of the movies are then saved in a CSV file (outputplots) together with the corresponding utility matrix (outputumatrix).

The other command will take the movie's descriptions and create an information retrieval system (Term Frequency, Inverse Document Frequency (TF-IDF) model) to allow the user to find movies typing some relevant words. This tf-idf model is then saved in the Django cache together with the initial recommendation systems models (CF item-based and log-likelihood ratio). The code is as follows:

각각의 uilitymatrix 파이렝 포함된 각각의 영화 제목들은 getplotfromomdb 펑션에 의해 영화 설명을 받아 올 수 있다.  이는 http://www.omdbapi.com/ 의 api 에 requests 를 보내는 것이다. 영화들의 설명, 제목은 csv 파일로 저장된다.

다른 커맨드는 영화의 설명을 가져오고 information retrieval system(tf,idf model) 을 만든다. 유저가 몇개의 단어를 통해 무비를 찾을 수 있게 . 이 tf-idf 모델은 초기 추천 시스템과 함께 캐싱된다.

```python
from django.core.management.base import BaseCommand
import os
import optparse
import numpy as np
import pandas as pd
import math
import json
import copy
from BeautifulSoup import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()
#nltk.download('stopwords')
stoplist = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from books_recsys_app.models import MovieData
from django.core.cache import cache

class Command(BaseCommand):

    option_list = BaseCommand.option_list + (
            optparse.make_option('-i', '--input', dest='input',
                                 type='string', action='store',
                                 help=('Input plots file')),
            optparse.make_option('--nmaxwords', '--nmaxwords', dest='nmaxwords',
                                 type='int', action='store',
                                 help=('nmaxwords')),
            optparse.make_option('--umatrixfile', '--umatrixfile', dest='umatrixfile',
                                 type='string', action='store',
                                 help=('umatrixfile')), 
        )
        
    def PreprocessTfidf(self,texts,stoplist=[],stem=False):
        newtexts = []
        for i in xrange(len(texts)):
            text = texts[i]
            if stem:
               tmp = [w for w in tknzr.tokenize(text) if w not in stoplist]
            else:
               tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
            newtexts.append(' '.join(tmp))
        return newtexts
    
    def handle(self, *args, **options):
        input_file = options['input']
        
        df = pd.read_csv(input_file)
        tot_textplots = df['plot'].tolist()
        tot_titles = df['title'].tolist()
        nmaxwords=options['nmaxwords']
        vectorizer = TfidfVectorizer(min_df=0,max_features=nmaxwords)
        processed_plots = self.PreprocessTfidf(tot_textplots,stoplist,True)
        mod_tfidf = vectorizer.fit(processed_plots)
        vec_tfidf = mod_tfidf.transform(processed_plots)
        ndims = len(mod_tfidf.get_feature_names())
        nmovies = len(tot_titles[:])
        
        #delete all data
        MovieData.objects.all().delete()
        
        matr = np.empty([1,ndims])
        titles = []
        cnt=0
        for m in xrange(nmovies):
            moviedata = MovieData()
            moviedata.title=tot_titles[m]
            moviedata.description=tot_textplots[m]
            moviedata.ndim= ndims
            moviedata.array=json.dumps(vec_tfidf[m].toarray()[0].tolist())
            moviedata.save()
            newrow = moviedata.array
            if cnt==0:
                matr[0]=newrow
            else:
                matr = np.vstack([matr, newrow])
            titles.append(moviedata.title)
            cnt+=1
        #cached
        cache.set('data', matr)
        cache.set('titles', titles)
        cache.set('model',mod_tfidf)

        
        #load the utility matrix
        umatrixfile = options['umatrixfile']
        df_umatrix = pd.read_csv(umatrixfile)
        Umatrix = df_umatrix.values[:,1:]
        cache.set('umatrix',Umatrix)
        #load rec methods... 
        cf_itembased = CF_itembased(Umatrix)
        cache.set('cf_itembased',cf_itembased)
        llr = LogLikelihood(Umatrix,titles)
        cache.set('loglikelihood',llr)
        
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine 
def sim(x,y,metric='cos'):
    if metric == 'cos':
       return 1.-cosine(x,y)
    else:#correlation
       return pearsonr(x,y)[0]
       
class CF_itembased(object):
...        
class LogLikelihood(object):
...

```
To run the command the syntax is:
```python
python manage.py load_data --input=plots.csv --nmaxwords=30000  --umatrixfile=umatrix.csv
```
The input parameter takes the movie's descriptions obtained using the get_plotsfromtitles command and creates a tf-idf model (see Chapter 4, Web-mining techniques) using a maximum of words specified by the nmaxwords parameter. The data of each movie is also saved in a MovieData object (title, tf-idf representation, description, and ndim number of words of the tf-idf vocabulary). Note that the first time the command is run the stopwords from nltk.download('stopwords') (commented in the preceding code) need to be downloaded.

The tf-idf model, the title's list, and the matrix of the tf-idf movies' representations, are saved in the Django cache using the commands:



```python
from django.core.cache import cache
...
cache.set('model',mod_tfidf)
cache.set('data', matr)
cache.set('titles', titles)
```
NOTE
Note that the cache Django module (django.core.cache) needs to be loaded (at the beginning of the file) to be used.

In the same way, the utility matrix (umatrixfile parameter) is used to initialize the two recommendation systems used by the application: item-based collaborative filtering and log-likelihood ratio method. Both methods are not written in the preceding code because they are essentially the same as the code described in Chapter 5, Recommendation systems (the full code can be seen in the chapter_7 folder of the author's GitHub repository as usual). The methods and the utility matrix are then loaded into the Django cache ready to use:
```python
cache.set('umatrix',Umatrix)
   cache.set('cf_itembased',cf_itembased)
   cache.set('loglikelihood',llr)
   ```
Now the data (and models) can be used in the web pages just by calling the corresponding name, as we will see in the following sections.